In [ ]:
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt
import seaborn as sns

f = open('/kaggle/input/swiggy-restaurants-dataset/data.json')
data = json.load(f)
f.close()

In [ ]:
Cities = list(data.keys())
len(Cities)

In [ ]:
c = Cities[443]

print('City Name :',c)

#print(data[c].keys())
print('Link :',data[c]['link'])
id = list(data[c]['restaurants'].keys())
print('First Restaurant id            :',id[0])
print('First Restaurant Name          :', data[c]['restaurants'][id[0]]['name'])
print('First Restaurant Rating        :', data[c]['restaurants'][id[0]]['rating'])
print('First Restaurant Rating_Counts :', data[c]['restaurants'][id[0]]['rating_count'])
print('First Restaurant Cost          :', data[c]['restaurants'][id[0]]['cost'])
print('First Restaurant Address       :', data[c]['restaurants'][id[0]]['address'])
print('First Restaurant Cuisine       :', data[c]['restaurants'][id[0]]['cuisine'])
print('First Restaurant License No.   :', data[c]['restaurants'][id[0]]['lic_no'])

Menu = list(data[c]['restaurants'][id[0]]['menu'].keys())
print('*'*45,data[c]['restaurants'][id[0]]['name'],',',c,'- Menu','*'*45)

for item in Menu:
    print(item)
    Sub_menu = list(data[c]['restaurants'][id[0]]['menu'][item].keys())
    for sub_item in Sub_menu :
        print(sub_item,'>>> Price :',data[c]['restaurants'][id[0]]['menu'][item][sub_item]['price'], end =' | ')
        print('veg_or_non_veg :',data[c]['restaurants'][id[0]]['menu'][item][sub_item]['veg_or_non_veg'])
    print('-'*100)
       
                                                          
print('First Restaurant Link          :',data[c]['restaurants'][id[0]]['link'])

In [ ]:
Data = pd.DataFrame()                               # Create an Empty Dataframe

for city in Cities:
    try:
        # Convert the city-wise restaurant data to pandas Dataframe
        DF = pd.DataFrame(data[city]['restaurants']).T  
        
        City = [city for i in range(DF.shape[0])]
        DF['City'] = City
        SubRegion = City                                          
        DF['SubRegion'] = SubRegion
        Data = pd.concat([Data,DF])
        
    except:
        
        Subregions = list(data[city].keys())
        for region in Subregions:
            
            
            DF = pd.DataFrame(data[city][region]['restaurants']).T
            City = [city for i in range(DF.shape[0])]
            
            DF['City'] = City
            
            SubRegion = [region for i in range(DF.shape[0])]
            
            DF['SubRegion'] = SubRegion
            Data = pd.concat([Data,DF])

In [ ]:
Data.shape

In [ ]:
def Menu(Data, i):
    Menu = list(Data.menu[i].keys())
    print('*'*35,Data['name'][i],',',Data['SubRegion'][i],Data['City'][i],'- Menu','*'*35)

    for item in Menu:
        print(item)
        Sub_menu = list(Data.menu[i][item].keys())
        for sub_item in Sub_menu :
            print(sub_item,'>>> Price :',Data.menu[i][item][sub_item]['price'], end =' | ')
            print('veg_or_non_veg :',Data.menu[i][item][sub_item]['veg_or_non_veg'])
        print('-'*100)
        
Menu(Data, 11223)

In [ ]:
Data.to_csv('Swiggy_data.csv')

In [ ]:
df = pd.read_csv('Swiggy_data.csv')

In [ ]:
df = df.rename(columns={'Unnamed: 0':'id', 'name':'Restaurant'})
df.drop_duplicates(subset='id', keep='first', inplace = True)
#df.set_index('id', inplace=True)
df.shape

In [ ]:
df = df.loc[:,['id','Restaurant','City','SubRegion','cost','cuisine','rating','rating_count','lic_no',
               'address','menu','link', 'type', 'price']]
df.head()

In [ ]:
#How many cities (including subregions) where Swiggy is having its restaurants listed?
df['SubRegion'].nunique()

In [ ]:
#How many cities (don't include subregions) where Swiggy is having its restaurants listed?
df['City'].nunique()

In [ ]:
#The Subregion of Delhi with the maximum number of restaurants listed on Swiggy?
d = df[df.City == 'Delhi']
d.groupby('SubRegion')['Restaurant'].count().sort_values(ascending=False).reset_index(name='Restaurants counts').head().style.background_gradient()

In [ ]:
Null = df.isnull().sum().reset_index().rename({0:'Null'}, axis = 1)
Null['% Null'] = round((Null['Null']/df.shape[0])*100, 0)
Null.sort_values(by='% Null', ascending = False).style.background_gradient()


In [ ]:
df = df.drop(['type','price'], axis =1)
df.dropna(inplace = True, axis = 0)
df.reset_index(inplace = True)
df.drop(['index'], axis = 1, inplace = True)
df.shape

In [ ]:
#List out the top 5 Restaurants with Maximum & minimum ratings throughout the dataset.
Rating = []
for i in range(df.shape[0]):
    try:
        Rating.append(float(df.rating[i]))     
    except:
        #print(Data.rating[i])
        if df.rating[i] == '--' or 'NA':
            Rating.append(np.NaN)                # Where rating is not givem keep it null
        else:
            print(df.rating[i])
print(len(Rating))
print(df.shape[0])
df.rating = Rating
df.rating.dtypes

In [ ]:
print('The maximum rating Restaurant are:')
df[df.rating ==df.rating.max()][['Restaurant', 'rating']].head(5)

In [ ]:
print('The minimum rating Restaurant are:')
df[df.rating==df.rating.min()][['Restaurant', 'rating']].head(5)

In [ ]:
#Name of top 5 cities with the highest number of restaurants listed.
B = df.City.value_counts(ascending=False).reset_index().rename({'index':'City', 'City':'Counts'}, axis=1)

B[B['Counts']<=B['Counts'].max()].head(5)

In [ ]:
import plotly.express as px
import plotly.offline as pyo

In [ ]:

#Top 10 cities as per the number of restaurants listed?
Top = df.City.value_counts(ascending=False).reset_index().rename({'index':'City', 'City':'Counts'}, axis=1).head(10)

fig =px.bar(Top,
            x= 'City',
            y= 'Counts',
            labels = {'Counts':'Restaurants Counts'},
            template='plotly_dark',
            color_discrete_sequence = px.colors.sequential.Viridis,
            title='<b>Top 10 Cities Restaurants listed')

fig.show()
pyo.plot(fig, filename = 'Top 10 Cities w.r.t Restaurants listed.html', auto_open = True)

In [ ]:
#Name the top 5 Most Popular Restaurants in Pune.
d = df[df.City == 'Pune']
df[df.rating==df.rating.max()][['Restaurant', 'rating']].head(5)

In [ ]:
#Which SubRegion in Delhi is having the least expensive restaurant in terms of cost?
d = df[df.City == 'Delhi']
df[df.cost==df.cost.min()][['SubRegion','cost']]

In [ ]:
#Top 5 most popular restaurant chains in India?
d = df.groupby('Restaurant', as_index=False).agg('count')[['Restaurant', 'id']].sort_values(by='id', ascending = False)

# Adjust the size and view of plot
plt.figure(figsize=(11,5), dpi=523)                                       
#sns.set_style('whitegrid') 

# Seaborn Bar plot
sns.barplot(data = d.head(),
           x='Restaurant',
           y = 'id',
           palette= "winter" 
          )     


# Set horizontal grid lines
plt.grid(True, axis = 'y', 
           color = 'blue', 
         linestyle = '--',  
         linewidth=0.1
        ) 
# Remove vertical grid lines
plt.grid(False, axis = 'x')                                   

# Remove boundry line, set Offset & trim the corner
sns.despine(offset=10, trim=False)

# Rotate the x tick value by 90 degree for better view
                                   
plt.xticks(rotation = 90)  
plt.xlabel("Restaurant Name", fontsize = 11)                             # Set X label         
plt.ylabel("Shop Counts", fontsize = 11)                                 # Set Y label
plt.title('Top 5 Restaurant Chain in India', fontsize = 15)              # Set title
plt.show()

In [ ]:
#Which restaurant in Pune has the most number of people visiting?
df.rating_count.unique()

In [ ]:
review = {}
for rating in df.rating_count.unique():
    if rating == 'Too Few Ratings':
        review[rating] = 0
    else:
        try:
            review[rating] = int(rating.split('+')[0])
        except:
            if rating.split('+')[0][-1] == 'K':
                review[rating] = int(rating.split('+')[0][:-1])*1000
review

In [ ]:
reviews = []
for i in range(df.shape[0]):
    reviews.append(review[df.rating_count[i]])
print(len(reviews))
print(df.shape[0])

df['Reviews'] = reviews

In [ ]:
d = df.sort_values(by = 'Reviews', ascending=False)[['Restaurant', 'City','rating','Reviews']]
d[d.City == 'Pune'].head(10)

In [ ]:
#Top 10 Restaurants with Maximum Ratings in Banglore
d = df[df.City == 'Bangalore']
d=df[df.rating==df.rating.max()][['Restaurant','rating']]
plt.figure(figsize=(11,5), dpi=523)                                       
#sns.set_style('whitegrid') 

# Seaborn Bar plot
sns.barplot(data = d.head(10),
           x='Restaurant',
           y = 'rating',
           palette= "summer" 
          )     


# Set horizontal grid lines
plt.grid(True, axis = 'y', 
           color = 'gray', 
         linestyle = '--',  
         linewidth=0.1
        ) 
# Remove vertical grid lines
plt.grid(False, axis = 'x')                                   

# Remove boundry line, set Offset & trim the corner
sns.despine(offset=10, trim=False)

# Rotate the x tick value by 90 degree for better view
                                   
plt.xticks(rotation = 90)  
plt.xlabel("Restaurant Name", fontsize = 11)                             # Set X label         
plt.ylabel("Ratings", fontsize = 11)                                 # Set Y label
plt.title('Top 10 Restaurant with Maximum Ratings in Banglore', fontsize = 15)              # Set title
plt.show()

In [ ]:
#Top 10 Restaurant in Patna w.r.t rating
Pune = df[df['City'] == 'Pune']
Pune[Pune['rating'] == Pune['rating'].max()][['Restaurant','City','rating']]



In [ ]:
Top = df.City.value_counts(ascending=False).reset_index().rename({'index':'City', 'City':'Counts'}, axis=1).head(10)

fig =px.bar(Pune.sort_values(by='rating',ascending = False).head(10),
            x= 'Restaurant',
            y= 'rating',
            labels = {'Restaurant':'Top 10 Restaurants in Pune','rating':'Rating'},
            template='plotly_dark',
            color_discrete_sequence = px.colors.sequential.Viridis,
            title='<b>Top 10 Restaurant in Pune w.r.t rating')

fig.show()

pyo.plot(fig, filename = 'Top 10 Restaurant in Pune w.r.t rating.html', auto_open = True)